In [1]:
import torch
import torch.nn as nn
import torchvision

In [3]:
# Load observations from the mnist dataset. The observations are divided into a training set and a test set
mnist_train = torchvision.datasets.MNIST('./data', train=True, download=True)
x_train = mnist_train.data.reshape(-1, 1, 28, 28).float()  # torch.functional.nn.conv2d argument must include channels (1)
y_train = torch.zeros((mnist_train.targets.shape[0], 10))  # Create output tensor
y_train[torch.arange(mnist_train.targets.shape[0]), mnist_train.targets] = 1  # Populate output

mnist_test = torchvision.datasets.MNIST('./data', train=False, download=True)
x_test = mnist_test.data.reshape(-1, 1, 28, 28).float()  # torch.functional.nn.conv2d argument must include channels (1)
y_test = torch.zeros((mnist_test.targets.shape[0], 10))  # Create output tensor
y_test[torch.arange(mnist_test.targets.shape[0]), mnist_test.targets] = 1  # Populate output

In [4]:
# Normalization of inputs
mean = x_train.mean()
std = x_train.std()
# Standard score = x - mean / std
x_train = (x_train - mean) / std
x_test = (x_test - mean) / std

# Divide training data into batches to speed up optimization
batches = 600
x_train_batches = torch.split(x_train, batches)
y_train_batches = torch.split(y_train, batches)

In [5]:
class SequentialCNNModel(nn.Module):
	def __init__(self):
		super(SequentialCNNModel, self).__init__()

		self.logits = nn.Sequential(nn.Conv2d(1, 32, 5, padding=2), nn.MaxPool2d(2), nn.Conv2d(32, 64, 5, padding=2), nn.MaxPool2d(2), nn.Flatten(), nn.Linear(64 * 7 * 7, 10))
	
	def f(self, x):
		return torch.softmax(self.logits(x), dim=1)
	
	def loss(self, x, y):
		return nn.functional.cross_entropy(self.logits(x), y.argmax(1))
	
	# Accuracy
	def accuracy(self, x, y):
		return torch.mean(torch.eq(self.f(x).argmax(1), y.argmax(1)).float())

In [7]:
model = SequentialCNNModel()

# Optimize: adjust W and b to minimize loss using stochastic gradient descent
optimizer = torch.optim.Adam(model.parameters(), 0.001)

In [8]:
for epoch in range(5):
    for batch in range(len(x_train_batches)):
        model.loss(x_train_batches[batch], y_train_batches[batch]).backward()  # Compute loss gradients
        optimizer.step()  # Perform optimization by adjusting W and b,
        optimizer.zero_grad()  # Clear gradients for next step

    print("accuracy = %s" % model.accuracy(x_test, y_test))

# 0.9857

/home/diderikk/.local/lib/python3.8/site-packages/torch/nn/functional.py:718: UserWarning: Named tensors and all their associated APIs are an experimental feature and subject to change. Please do not use them for anything important until they are released as stable. (Triggered internally at  /pytorch/c10/core/TensorImpl.h:1156.)
  return torch.max_pool2d(input, kernel_size, stride, padding, dilation, ceil_mode)


accuracy = tensor(0.9721)
accuracy = tensor(0.9803)
accuracy = tensor(0.9820)
accuracy = tensor(0.9836)
accuracy = tensor(0.9857)
